# Algoritmos populares de agrupación / Clustering Algorithms

## Visualizando datos de COVID-19

Actualización a julio de 2024

In [ ]:
# Dependencies
import pandas as pd
pd.set_option('display.max_columns', 100)
from pandas.plotting import scatter_matrix
import numpy as np
import math
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans

import warnings
warnings.filterwarnings('ignore')

In [ ]:
#!pip install pip
#!pip install --upgrade seaborn
#!pip install --upgrade numpy

In [ ]:
# Opciones de visualizació de cifras:
pd.options.display.float_format = '{:,.2f}'.format #'${:,.2f}'

## Descarga de datos

Datos reportados por la Secretaría de Salud sobre la COVID19. Los datos se publican diariamente y se pueden encontrar aquí.

Los datos están en formato zip, dentro del archivo comprimido hay un archivo de tipo CSV.

Los utilizaremoss sin descargarlos y descomprimirlos.

Fuente: https://www.gob.mx/salud/documentos/datos-abiertos-bases-historicas-direccion-general-de-epidemiologia?idiom=es

In [ ]:
# Cara renglón en este data set representa a una persona

df_2025 = pd.read_csv('https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2025/05/datos_abiertos_influenza_covid19_06.05.2025.zip', 
                  compression = 'zip', encoding = 'latin1')

df_2025.head()

In [ ]:
#

df_2025.tail()

In [ ]:
#

df_2025['CLASIFICACION_FINAL'] = df_2025['CLASIFICACION_FINAL_COVID']

#df_2025['CLASIFICACION_FINAL'] = df_2025['CLASIFICACION_FINAL_FLU']

In [ ]:
# Cara renglón en este data set representa a una persona

df_2024 = pd.read_csv('https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2024/12/datos_abiertos_influenza_covid19_31.12.2024.zip', 
                  compression = 'zip', encoding = 'latin1')

df_2024.head()

In [ ]:
#

df_2024.tail()

In [ ]:
#

df_2024['CLASIFICACION_FINAL'] = df_2024['CLASIFICACION_FINAL_COVID']

#df_2024['CLASIFICACION_FINAL'] = df_2024['CLASIFICACION_FINAL_FLU']

In [ ]:
# Cara renglón en este data set representa a una persona

df_2023 = pd.read_csv('https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2023/COVID19MEXICO2023.zip', 
                  compression = 'zip', encoding = 'latin1')

df_2023.head()

In [ ]:
#

df_2023.tail()

In [ ]:
# Cara renglón en este data set representa a una persona

df_2022 = pd.read_csv('https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2022/COVID19MEXICO2022.zip', 
                  compression = 'zip', encoding = 'latin1')

df_2022.head()

In [ ]:
#

df_2022.tail()

In [ ]:
# Cara renglón en este data set representa a una persona

df_2021 = pd.read_csv('https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2021/COVID19MEXICO2021.zip', 
                  compression = 'zip', encoding = 'latin1')

df_2021.head()

In [ ]:
#

df_2021.tail()

In [ ]:
# Cara renglón en este data set representa a una persona

df_2020 = pd.read_csv('https://datosabiertos.salud.gob.mx/gobmx/salud/datos_abiertos/historicos/2020/COVID19MEXICO2020.zip', 
                  compression = 'zip', encoding = 'latin1')

df_2020.head()

In [ ]:
#

df_2020.tail()

In [ ]:
# Número de datos

df_2025.shape, df_2024.shape, df_2023.shape, df_2022.shape, df_2021.shape, df_2020.shape

In [ ]:
# Concatenamos la información

df = pd.concat([ df_2020, df_2021, df_2022, df_2023, df_2024, df_2025 ], ignore_index = True )

df.head()

In [ ]:
# Número de datos

df.shape, 248602 + 175586 + 1304796 + 6451944 + 8830345 + 3868396

In [ ]:
# Nombres de las columnas

df.columns

In [ ]:
# Tipo de caso que analizaremos y cómo los clasificaremos

df['CLASIFICACION_FINAL'].value_counts()

In [ ]:
df[ df['CLASIFICACION_FINAL'].isna() ]

## Claves de la 'CLASIFICACION FINAL'


* 1 = CASO DE COVID-19 CONFIRMADO POR ASOCIACIÓN CLÍNICA EPIDEMIOLÓGICA: aplica cuando el caso informó ser contacto de un positivo a COVID-19 y al caso no se le tomo muestra o la muestra resultó no válida
* 2 = CASO DE COVID-19 CONFIRMADO POR COMITÉ DE  DICTAMINACIÓN: aplica para defunciones que no se le tomó muestra o sí se tomó muestra, pero la muestra resultó no válida
* 3 = CASO DE SARS-COV-2 CONFIRMADO POR LABORATORIO: aplica  cuanndo el caso tiene muestra y resultó positiva a SARS-CoV-2, sin importar si el caso tienen asociación clínica epidemiológica
* 4 = INVÁLIDO POR LABORATORIO: aplica cuando el caso no tienen asociación clínico epidemiológica, ni dictaminación a COVID-19, se le tomó muestra y esta resultó no válida
* 5 = NO REALIZADO POR LABORATORIO: aplica cuando el caso no tienen asociación clínico epidemiológica, ni dictaminación a COVID-19 y se le tomó muestra y esta no se procesó
* 6 = CASO SOSPECHOSO: aplica cuando el caso no tienen asociación clínico epidemiológica, ni dictaminación a COVID-19 y no se le tomó muestra, o se le tomó muestra y está pendiente de resultado, sin importar otra condición
* 7 = NEGATIVO A SARS-COV-2 POR LABORATORIO: aplica cuando el caso se le tomo muestra y esta resultó negativa a SARS-COV-2 o positiva a cualquier otro virus respiratorio (Influenza, VSR, Bocavirus, otros) sin importar que este caso tenga asociación clínico epidemiológica o dictaminación a COVID-19

## Clasificando los casos de acuerdo a su calidad de confirmados, sospechosos o negativos [tablero olvidado por la 4t]

Para checar consistencia: https://datos.covid-19.conacyt.mx/

## Casos negativos

* 7 = NEGATIVO A SARS-COV-2 POR LABORATORIO: aplica cuando el caso se le tomo muestra y esta resultó negativa a SARS-COV-2 o positiva a cualquier otro virus respiratorio (Influenza, VSR, Bocavirus, otros) sin importar que este caso tenga asociación clínico epidemiológica o dictaminación a COVID-19

In [ ]:
# 

Negativos = df[ df["CLASIFICACION_FINAL"] == 7 ]

Negativos.shape

## Casos confirmados

* 1 = CASO DE COVID-19 CONFIRMADO POR ASOCIACIÓN CLÍNICA EPIDEMIOLÓGICA: aplica cuando el caso informó ser contacto de un positivo a COVID-19 y al caso no se le tomo muestra o la muestra resultó no válida
* 2 = CASO DE COVID-19 CONFIRMADO POR COMITÉ DE  DICTAMINACIÓN: aplica para defunciones que no se le tomó muestra o sí se tomó muestra, pero la muestra resultó no válida
* 3 = CASO DE SARS-COV-2 CONFIRMADO POR LABORATORIO: aplica  cuanndo el caso tiene muestra y resultó positiva a SARS-CoV-2, sin importar si el caso tienen asociación clínica epidemiológica

In [ ]:
#

Confirmados = df[ df["CLASIFICACION_FINAL"].isin( [1, 2, 3] ) ]

Confirmados.shape

Gráfica - Histograma por sexo

In [ ]:
# CLAVE DESCRIPCIÓN
# 1 - MUJER
# 2 - HOMBRE

Confirmados['SEXO'].value_counts()

In [ ]:
Confirmados[Confirmados['SEXO'] == 2]['EDAD']

In [ ]:
# Histograma:

H = Confirmados[Confirmados['SEXO'] == 2]['EDAD']

M = Confirmados[Confirmados['SEXO'] == 1]['EDAD']

plt.figure(figsize=(10, 6))  

# Añadir los histogramas. alpha es para la opacidad.
plt.hist(H, bins=30, alpha=0.75, label='Hombres')
plt.hist(M, bins=30, alpha=0.75, label='Mujeres')

# Configurar las leyendas
plt.legend(loc = 'upper right')

# Título y etiquetas
plt.title('Casos confirmados acumulados por edad', fontsize=16, fontname='Times New Roman', pad=20)
plt.xlabel('Edad', fontsize=14, fontname='Times New Roman')
plt.ylabel('Frecuencia', fontsize=14, fontname='Times New Roman')

plt.show()

In [ ]:
#

Confirmados["FECHA_SINTOMAS"].value_counts()

In [ ]:
#

Confirmados["FECHA_INGRESO"].value_counts()

## Casos sospechosos

* 6 = CASO SOSPECHOSO: aplica cuando el caso no tienen asociación clínico epidemiológica, ni dictaminación a COVID-19 y no se le tomó muestra, o se le tomó muestra y está pendiente de resultado, sin importar otra condición

In [ ]:
#

Sospechosos = df[ df["CLASIFICACION_FINAL"] == 6 ]

Sospechosos.shape 

## Defunciones

La base de datos no cuenta con una columna que indique de forma categórica si el caso falleció o no. 

Podemos obtener este dato de forma indirecta; utillizando la fecha de defunción en "FECHA_DEF"

In [ ]:
#

df["FECHA_DEF"].value_counts()

In [ ]:
#
#Confirmados = df[ df["CLASIFICACION_FINAL"].isin( [1, 2, 3] ) ]


Defunciones = df[ ( df["CLASIFICACION_FINAL"].isin( [1, 2, 3] ) ) & ( df["FECHA_DEF"] != "9999-99-99"  ) ]
Defunciones.shape 

Gráfica - Histograma por sexo

In [ ]:
# CLAVE	DESCRIPCIÓN
# 1 - MUJER
# 2 - HOMBRE

Defunciones['SEXO'].value_counts()

In [ ]:
# Histograma:

H = Defunciones[(Defunciones['SEXO'] == 2) ]['EDAD']

M = Defunciones[(Defunciones['SEXO'] == 1) ]['EDAD']

plt.figure(figsize=(10, 6))  

# Añadir los histogramas. alpha es para la opacidad.
plt.hist(H, bins=30, alpha=0.75, label='Hombres')
plt.hist(M, bins=30, alpha=0.75, label='Mujeres')

# Configurar las leyendas
plt.legend(loc = 'upper right')

# Título y etiquetas
plt.title('Defunciones acumuladas por edad (TODA LA PANDEMIA y hasta el momento)', fontsize=16, fontname='Times New Roman', pad=20)
plt.xlabel('Edad', fontsize=14, fontname='Times New Roman')
plt.ylabel('Frecuencia', fontsize=14, fontname='Times New Roman')

plt.show()

## Serie de casos confirmados diarios

In [ ]:
# Agrupando por fechas: 

Grupos_fecha = Confirmados.groupby("FECHA_SINTOMAS")

# Guardamos como Data Frame

Serie_Confirmados = Grupos_fecha.size().to_frame("Confirmados").reset_index()

# Convertimos a tipo de dato fecha la columna:

Serie_Confirmados["Fecha"] = pd.to_datetime(Serie_Confirmados["FECHA_SINTOMAS"])

Serie_Confirmados.head()

In [ ]:
# Agrupando por fechas: 

Grupos_fecha = Confirmados.groupby("FECHA_INGRESO")

# Guardamos como Data Frame

Serie_Confirmados_B = Grupos_fecha.size().to_frame("Confirmados").reset_index()

Serie_Confirmados_B = Serie_Confirmados_B.rename(columns = {'Confirmados': 'Confirmados_Ing'})

# Convertimos a tipo de dato fecha la columna:

Serie_Confirmados_B["Fecha"] = pd.to_datetime(Serie_Confirmados_B["FECHA_INGRESO"])

Serie_Confirmados_B.head()

## Serie de defunciones diarias

### A. Por fecha de sintomas

In [ ]:
# Agrupando por fechas: 

Grupos_fecha = Defunciones.groupby("FECHA_SINTOMAS")

# Guardamos como Data Frame

Serie_Defunciones_A = Grupos_fecha.size().to_frame("Defunciones").reset_index()

# Convertimos a tipo de dato fecha la columna:

Serie_Defunciones_A["Fecha"] = pd.to_datetime(Serie_Defunciones_A["FECHA_SINTOMAS"])

Serie_Defunciones_A.head()

### B. Por fecha de defunción

In [ ]:
# Agrupando por fechas: 

Grupos_fecha = Defunciones.groupby("FECHA_DEF")

# Guardamos como Data Frame

Serie_Defunciones_B = Grupos_fecha.size().to_frame("Defunciones").reset_index()

# Convertimos a tipo de dato fecha la columna:

Serie_Defunciones_B["Fecha"] = pd.to_datetime(Serie_Defunciones_B["FECHA_DEF"])

Serie_Defunciones_B.head()

## Serie de casos sospechosos diarios

In [ ]:
# Agrupando por fechas: 

Grupos_fecha = Sospechosos.groupby("FECHA_SINTOMAS")

# Guardamos como Data Frame

Serie_Sospechosos = Grupos_fecha.size().to_frame("Sospechosos").reset_index()

# Convertimos a tipo de dato fecha la columna:

Serie_Sospechosos["Fecha"] = pd.to_datetime(Serie_Sospechosos["FECHA_SINTOMAS"])

Serie_Sospechosos.head()

## Graficando 

In [ ]:
# Preparamos DataFrame's

conf = Serie_Confirmados[['Confirmados', 'Fecha']] # Fecha de síntomas

conf_b = Serie_Confirmados_B[['Confirmados_Ing', 'Fecha']] # Fecha de ingreso

defun_a = Serie_Defunciones_A[['Defunciones', 'Fecha']] # Fecha de síntomas

defun_b = Serie_Defunciones_B[['Defunciones', 'Fecha']] # Fecha de defunción

sosp = Serie_Sospechosos[['Sospechosos', 'Fecha']] # Fecha de síntomas

In [ ]:
# Unimos las series: 

# Confirmados + Confirmados (con fecha de ingreso)
Series_Casos = pd.merge(conf, conf_b, how = 'outer', on = 'Fecha')

# Confirmados + Defunciones (con fecha de síntomas)
Series_Casos = pd.merge(Series_Casos, defun_a, how = 'outer', on = 'Fecha')

# Renombramos serie defunciones por fecha de síntomas
Series_Casos = Series_Casos.rename(columns = {'Defunciones': 'Defunciones_Sint'})

# Confirmados + Defunciones (con fecha de síntomas) + Defunciones (con fecha de muerte)
Series_Casos = pd.merge(Series_Casos, defun_b, how = 'outer', on = 'Fecha')

# Renombramos serie defunciones por fecha de muerte
Series_Casos = Series_Casos.rename(columns = {'Defunciones': 'Defunciones_Muer'})

# Confirmados + Defunciones (con fecha de síntomas) + Defunciones (con fecha de muerte) + Sospechosos
Series_Casos = pd.merge(Series_Casos, sosp, how = 'outer', on = 'Fecha')

#Series_Casos = Series_Casos.sort_values(by = 'Fecha', ascending = False)

Series_Casos.head()

In [ ]:
# Ordenamos y reemplazamos nulos por ceros (0)

#Series_Casos = Series_Casos.fillna(0)

Series_Casos = Series_Casos.sort_values(by = 'Fecha', ascending = True).reset_index()

Series_Casos = Series_Casos[['Fecha', 'Confirmados', 'Confirmados_Ing', 'Sospechosos', 'Defunciones_Sint', 'Defunciones_Muer']]

Series_Casos

In [ ]:
# Series TODAS

plt.figure(figsize=(10, 6)) 

# Agrega cada serie de datos como una línea en el gráfico
plt.plot(Series_Casos['Fecha'], Series_Casos['Confirmados'], label='Confirmados', color='darkblue')
plt.plot(Series_Casos['Fecha'], Series_Casos['Sospechosos'], label='Sospechosos', color='orange')
plt.plot(Series_Casos['Fecha'], Series_Casos['Defunciones_Sint'], label='Defunciones por fecha de síntomas', color='darkred')
plt.plot(Series_Casos['Fecha'], Series_Casos['Defunciones_Muer'], label='Defunciones por fecha de muerte', color='darkgreen')

# Configura el título y las leyendas
plt.title('Casos Confirmados y Sospechosos, y Defunciones (por fecha de síntomas y defunción)')
plt.legend()

# Formato de la fecha en el eje X
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.AutoDateLocator())
plt.xticks(rotation=0)  # Rota las fechas para mejor lectura

# Muestra el gráfico
plt.tight_layout()  
plt.show()

In [ ]:
# Series TODAS - Medias Móviles

plt.figure(figsize=(10, 6)) 

# Agrega cada serie de datos como una línea en el gráfico
plt.plot(Series_Casos['Fecha'], Series_Casos['Confirmados'].rolling(window = 7).mean()/1000, label='Confirmados', color='darkblue')
plt.plot(Series_Casos['Fecha'], Series_Casos['Sospechosos'].rolling(window = 7).mean()/1000, label='Sospechosos', color='orange')
plt.plot(Series_Casos['Fecha'], Series_Casos['Defunciones_Sint'].rolling(window = 7).mean()/1000, label='Defunciones por fecha de síntomas', color='darkred')
plt.plot(Series_Casos['Fecha'], Series_Casos['Defunciones_Muer'].rolling(window = 7).mean()/1000, label='Defunciones por fecha de muerte', color='darkgreen')

# Configura el título y las leyendas
plt.title('Casos Confirmados y Sospechosos, y Defunciones (por fecha de síntomas y defunción) (en miles)')
plt.legend()

# Formato de la fecha en el eje X
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.AutoDateLocator())
plt.xticks(rotation=0)  # Rota las fechas para mejor lectura

# Muestra el gráfico
plt.tight_layout()  
plt.show()

In [ ]:
# Series Confirmados

plt.figure(figsize=(10, 6)) 

# Agrega cada serie de datos como una línea en el gráfico
plt.plot(Series_Casos['Fecha'], Series_Casos['Confirmados']/1000, label='Confirmados', color='darkblue')

# Configura el título y las leyendas
plt.title('Evolución de los casos confirmados (por fecha de síntomas) - en miles')
#plt.legend()

# Formato de la fecha en el eje X
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.AutoDateLocator())
plt.xticks(rotation=0)  # Rota las fechas para mejor lectura

# Muestra el gráfico
plt.tight_layout()  
plt.show()

In [ ]:
# Series Confirmados Medias Móviles

plt.figure(figsize=(10, 6)) 

# Agrega cada serie de datos como una línea en el gráfico
plt.plot(Series_Casos['Fecha'], Series_Casos['Confirmados'].rolling(window = 7).mean()/1000, label='Confirmados', color='darkblue')

# Configura el título y las leyendas
plt.title('Evolución de los casos confirmados (por fecha de síntomas y media mólvil 7 días) - en miles')
#plt.legend()

# Formato de la fecha en el eje X
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.AutoDateLocator())
plt.xticks(rotation=45)  # Rota las fechas para mejor lectura

# Muestra el gráfico
plt.tight_layout()  
plt.show()

In [ ]:
# Series Confirmados ACUMULADOS

plt.figure(figsize=(10, 6)) 

# Agrega cada serie de datos como una línea en el gráfico
plt.plot(Series_Casos['Fecha'], Series_Casos['Confirmados'].fillna(0).cumsum()/1000000, label='Confirmados', color='darkblue')

# Configura el título y las leyendas
plt.title('Casos confirmados acumulados (por fecha de síntomas) - en millones')
#plt.legend()

# Formato de la fecha en el eje X
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.AutoDateLocator())
plt.xticks(rotation=45)  # Rota las fechas para mejor lectura

# Muestra el gráfico
plt.tight_layout()  
plt.show()

In [ ]:
# Series Defunciones (fecha síntomas y fecha de defunción)

plt.figure(figsize=(10, 6)) 

# Agrega cada serie de datos como una línea en el gráfico
plt.plot(Series_Casos['Fecha'], Series_Casos['Defunciones_Sint']/1000, label='Defunciones por fecha de síntomas', color='darkred')
plt.plot(Series_Casos['Fecha'], Series_Casos['Defunciones_Muer']/1000, label='Defunciones por fecha de muerte', color='darkgreen')

# Configura el título y las leyendas
plt.title('Evolución de las defunciones (por fecha de síntomas y por fecha de defunción) - en miles')
plt.legend()

# Formato de la fecha en el eje X
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.AutoDateLocator())
plt.xticks(rotation=0)  # Rota las fechas para mejor lectura

# Muestra el gráfico
plt.tight_layout()  
plt.show()

In [ ]:
# Series Defunciones Medias Móviles

plt.figure(figsize=(10, 6)) 

# Agrega cada serie de datos como una línea en el gráfico
plt.plot(Series_Casos['Fecha'], Series_Casos['Defunciones_Sint'].rolling(window = 7).mean()/1000, label='Defunciones por fecha de síntomas', color='darkred')
plt.plot(Series_Casos['Fecha'], Series_Casos['Defunciones_Muer'].rolling(window = 7).mean()/1000, label='Defunciones por fecha de muerte', color='darkgreen')

# Configura el título y las leyendas
plt.title('Evolución de las defunciones (por fecha de síntomas y por fecha de defunción y media móvil 7 días) - en miles')
plt.legend()

# Formato de la fecha en el eje X
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.AutoDateLocator())
plt.xticks(rotation=0)  # Rota las fechas para mejor lectura

# Muestra el gráfico
plt.tight_layout()  
plt.show()

In [ ]:
# Series Defunciones TODAS ACUMULADAS

plt.figure(figsize=(10, 6)) 

# Agrega cada serie de datos como una línea en el gráfico
plt.plot(Series_Casos['Fecha'], Series_Casos['Defunciones_Sint'].fillna(0).cumsum()/1000, label='Defunciones por fecha de síntomas', color='darkred')
plt.plot(Series_Casos['Fecha'], Series_Casos['Defunciones_Muer'].fillna(0).cumsum()/1000, label='Defunciones por fecha de muerte', color='darkgreen')

# Configura el título y las leyendas
plt.title('Defunciones acumuladas (por fecha de síntomas y por fecha de defunción) - en miles')
plt.legend()

# Formato de la fecha en el eje X
plt.gca().xaxis.set_major_formatter(plt.matplotlib.dates.DateFormatter('%Y-%m-%d'))
plt.gca().xaxis.set_major_locator(plt.matplotlib.dates.AutoDateLocator())
plt.xticks(rotation=0)  # Rota las fechas para mejor lectura

# Muestra el gráfico
plt.tight_layout()  
plt.show()

In [ ]:
#

Series_Casos.head()

In [ ]:
#

Series_Casos.tail()

In [ ]:
# To CSV:

#Series_Casos.to_csv('Series_Casos.csv', index = False, encoding = 'utf-8-sig')

## Algoritmos de agrupamiento (clustering)

Sabemos que los algoritmos pertenecientes a la familia del Aprendizaje No Supervisado no tienen ninguna variable que predecir ligada a los datos. En lugar de tener una salida, los datos solo tienen una entrada que serían múltiples variables que describen los datos. Aquí es donde entra en juego la agrupación.

Clustering es la tarea de agrupar un conjunto de objetos de manera que los objetos del mismo grupo sean más similares entre sí que a los objetos de otros grupos. La similitud es una métrica que refleja la fuerza de la relación entre dos objetos de datos. 

La agrupación en clústeres se utiliza principalmente (pero no exclusivamente) para la exploración de datos. Tiene múltiples usos en muchos campos, como el machine learning, el reconocimiento de patrones, el análisis de imágenes, la recuperación de información, la bioinformática, la compresión de datos y los gráficos por computadora.

Hacemos un repaso de esas técnicas.

### K-medias

Existen muchos modelos de agrupación. Revisaremos los más populares. A pesar de su simplicidad, K-means se utiliza ampliamente para la agrupación en clústeres en muchas aplicaciones de ciencia de datos, lo que es especialmente útil si necesita descubrir rápidamente información a partir de datos sin etiquetar. En este notebook, vemos cómo utilizar k-Means para la segmentación de casos.

In [ ]:
#Confirmados, Sospechosos, Defunciones
# Definición de catalógos:
ESTADO = ['AGUASCALIENTES', 'BAJA CALIFORNIA', 'BAJA CALIFORNIA SUR', 'CAMPECHE', 'COAHUILA DE ZARAGOZA', 
          'COLIMA', 'CHIAPAS', 'CHIHUAHUA', 'CIUDAD DE MÉXICO', 'DURANGO', 'GUANAJUATO', 'GUERRERO', 'HIDALGO', 
          'JALISCO', 'MÉXICO', 'MICHOACÁN DE OCAMPO', 'MORELOS', 'NAYARIT', 'NUEVO LEÓN', 'OAXACA', 'PUEBLA', 
          'QUERÉTARO', 'QUINTANA ROO', 'SAN LUIS POTOSÍ', 'SINALOA', 'SONORA', 'TABASCO', 'TAMAULIPAS', 
          'TLAXCALA', 'VERACRUZ DE IGNACIO DE LA LLAVE', 'YUCATÁN', 'ZACATECAS']

ABREVIATURA = ['AS', 'BC', 'BS', 'CC', 'CL', 'CM', 'CS', 'CH', 'DF', 'DG', 'GT', 'GR', 'HG', 'JC', 'MC', 'MN',
               'MS', 'NT', 'NL', 'OC', 'PL', 'QT', 'QR', 'SP', 'SL', 'SR', 'TC', 'TS', 'TL', 'VZ', 'YN', 'ZS']
#

In [ ]:
#

Defunciones_KM = Defunciones

Defunciones_KM.shape

In [ ]:
# Variable de Días

Defunciones_KM['Dias_Ing'] = ( pd.to_timedelta( 
                        pd.to_datetime( Defunciones_KM['FECHA_INGRESO'], format = '%Y-%m-%d' ) - \
                        pd.to_datetime( Defunciones_KM['FECHA_SINTOMAS'], format = '%Y-%m-%d' ) 
                   ).dt.total_seconds() / 3600 ) / 24

Defunciones_KM['Dias_Def'] = ( pd.to_timedelta( 
                        pd.to_datetime( Defunciones_KM['FECHA_DEF'], format = '%Y-%m-%d' ) - \
                        pd.to_datetime( Defunciones_KM['FECHA_SINTOMAS'], format = '%Y-%m-%d' ) 
                   ).dt.total_seconds() / 3600 ) / 24

In [ ]:
#

Defunciones_KM.info()

In [ ]:
# Pienso que estas son las variables relevantes de la tabla:
# 'ENTIDAD_UM', 'ENTIDAD_RES', 'SEXO', 'EDAD', 'EMBARAZO', 'INDIGENA', 
# 'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM', 
# 'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
# 'UCI', 'INTUBADO', 'Dias_Ing', 'Dias_Def'

Defunciones_KM.head()

In [ ]:
# 

Defunciones_KM.ENTIDAD_UM.value_counts(normalize = True)

In [ ]:
# 

Defunciones_KM.ENTIDAD_RES.value_counts(normalize=True)

In [ ]:
# 

Defunciones_KM.SEXO.value_counts(normalize = True)

In [ ]:
# 

Defunciones_KM.EDAD.value_counts(normalize = True)

In [ ]:
#

Defunciones_KM = Defunciones_KM[ Defunciones_KM['EDAD'] <= 100 ]

Defunciones_KM.shape

In [ ]:
# 

Defunciones_KM.EMBARAZO.value_counts(normalize = True)

In [ ]:
# 

Defunciones_KM.INDIGENA.value_counts(normalize = True)

In [ ]:
#

Defunciones_KM = Defunciones_KM[ Defunciones_KM["INDIGENA"].isin( [1, 2] ) ]

Defunciones_KM.shape 

In [ ]:
# 

Defunciones_KM.DIABETES.value_counts(normalize = True)

In [ ]:
#

Defunciones_KM = Defunciones_KM[ Defunciones_KM["DIABETES"].isin( [1, 2] ) ]

Defunciones_KM.shape 

In [ ]:
# 

Defunciones_KM.EPOC.value_counts(normalize = True)

In [ ]:
#

Defunciones_KM = Defunciones_KM[ Defunciones_KM["EPOC"].isin( [1, 2] ) ]

Defunciones_KM.shape 

In [ ]:
# 

Defunciones_KM.ASMA.value_counts(normalize = True)

In [ ]:
#

Defunciones_KM = Defunciones_KM[ Defunciones_KM["ASMA"].isin( [1, 2] ) ]

Defunciones_KM.shape 

In [ ]:
# 

Defunciones_KM.INMUSUPR.value_counts(normalize = True)

In [ ]:
#

Defunciones_KM = Defunciones_KM[ Defunciones_KM["INMUSUPR"].isin( [1, 2] ) ]

Defunciones_KM.shape 

In [ ]:
# 

Defunciones_KM.HIPERTENSION.value_counts(normalize = True)

In [ ]:
#

Defunciones_KM = Defunciones_KM[ Defunciones_KM["HIPERTENSION"].isin( [1, 2] ) ]

Defunciones_KM.shape 

In [ ]:
# 

Defunciones_KM.OTRA_COM.value_counts(normalize = True)

In [ ]:
#

Defunciones_KM = Defunciones_KM[ Defunciones_KM["OTRA_COM"].isin( [1, 2] ) ]

Defunciones_KM.shape 

In [ ]:
# 

Defunciones_KM.CARDIOVASCULAR.value_counts(normalize = True)

In [ ]:
#

Defunciones_KM = Defunciones_KM[ Defunciones_KM["CARDIOVASCULAR"].isin( [1, 2] ) ]

Defunciones_KM.shape 

In [ ]:
# 

Defunciones_KM.OBESIDAD.value_counts(normalize = True)

In [ ]:
#

Defunciones_KM = Defunciones_KM[ Defunciones_KM["OBESIDAD"].isin( [1, 2] ) ]

Defunciones_KM.shape 

In [ ]:
# 

Defunciones_KM.RENAL_CRONICA.value_counts(normalize = True)

In [ ]:
#

Defunciones_KM = Defunciones_KM[ Defunciones_KM["RENAL_CRONICA"].isin( [1, 2] ) ]

Defunciones_KM.shape 

In [ ]:
# 

Defunciones_KM.TABAQUISMO.value_counts(normalize = True)

In [ ]:
#

Defunciones_KM = Defunciones_KM[ Defunciones_KM["TABAQUISMO"].isin( [1, 2] ) ]

Defunciones_KM.shape 

In [ ]:
# 

Defunciones_KM.UCI.value_counts(normalize = True)

In [ ]:
#

Defunciones_KM = Defunciones_KM[ Defunciones_KM["UCI"].isin( [1, 2] ) ]

Defunciones_KM.shape 

In [ ]:
# 

Defunciones_KM.INTUBADO.value_counts(normalize = True)

In [ ]:
#

Defunciones_KM = Defunciones_KM[ Defunciones_KM["INTUBADO"].isin( [1, 2] ) ]

Defunciones_KM.shape 

In [ ]:
# 

Defunciones_KM[ ['Dias_Ing', 'Dias_Def']].describe()

In [ ]:
#

Defunciones_KM = Defunciones_KM[ ( Defunciones_KM['Dias_Ing'] >= 0 ) & ( Defunciones_KM['Dias_Ing'] <= 20 ) & \
                                 ( Defunciones_KM['Dias_Def'] >= 0 ) & ( Defunciones_KM['Dias_Def'] <= 60 ) ]

Defunciones_KM.shape

In [ ]:
# 

Defunciones_KM[ ['Dias_Ing', 'Dias_Def']].describe()

In [ ]:
#
# Guarda el DataFrame en un archivo CSV
Defunciones_KM.to_csv('Defunciones_KM.csv', index=False)

## Leer tabla de datos de defunciones

In [ ]:
# Leer el archivo CSV y guardarlo en un DataFrame

Defunciones_KM = pd.read_csv('Defunciones_KM.csv')

In [ ]:
# Let's see our data in a detailed way with pairplot

sns.pairplot( Defunciones_KM[ [ 'SEXO','EDAD', 'Dias_Ing', 'Dias_Def' ] ] , hue = 'SEXO' )

plt.show()

In [ ]:
# Let's see our data in a detailed way with pairplot

sns.pairplot( Defunciones_KM[ [ 'DIABETES','EDAD', 'Dias_Ing', 'Dias_Def' ] ] , hue = 'DIABETES' )

plt.show()

In [ ]:
# Let's see our data in a detailed way with pairplot

sns.pairplot( Defunciones_KM[ [ 'HIPERTENSION','EDAD', 'Dias_Ing', 'Dias_Def' ] ] , hue = 'HIPERTENSION' )

plt.show()

De los gráficos anteriores vemos que el género y otras variables no parecen tener relación directa con la segmentación de defunciones. Es por eso que podemos dejarlo y continuar con otras características, razón por la cual usaremos el parámetro X de ahora en adelante.

In [ ]:
# Definimos X:

X = Defunciones_KM [ [ 'ID_REGISTRO', 'ENTIDAD_UM', 'ENTIDAD_RES', 'SEXO', 'EDAD', 'INDIGENA', 
                       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM', 
                       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
                       'UCI', 'INTUBADO', 'Dias_Ing', 'Dias_Def' ] ]

In [ ]:
# 

km = KMeans( n_clusters = 3 ).fit( X.drop(['ID_REGISTRO'], axis = 1) )

km.inertia_ #km.labels_

In [ ]:
#

clusters = []

#
for i in range(1, 11):
    km = KMeans( n_clusters = i ).fit( X.drop(['ID_REGISTRO'], axis = 1) )
    clusters.append( km.inertia_ )

#
fig, ax = plt.subplots( figsize = (12, 8) )
sns.lineplot(x = list(range(1, 11)), y = clusters, ax = ax)
ax.set_title('Searching for Elbow')
ax.set_xlabel('Clusters')
ax.set_ylabel('Inertia')

# Annotate arrow
ax.annotate('Possible Elbow Point', xy = (2, 91797400), xytext = (2, 110000000), xycoords = 'data',          
             arrowprops = dict( arrowstyle = '->', connectionstyle = 'arc3', color = 'blue', lw = 2))

ax.annotate('Possible Elbow Point', xy = (3, 72737691), xytext = (3, 90000000), xycoords = 'data',          
             arrowprops = dict( arrowstyle = '->', connectionstyle = 'arc3', color = 'blue', lw = 2))

plt.show()

El método del codo nos dice que seleccionemos el grupo cuando hay un cambio significativo en la inercia. Como podemos ver en el gráfico, podemos decir que puede ser 2, 3 o 6. Veamos ambos resultados en el gráfico y decidamos.

In [ ]:
# 2 cluster
km2 = KMeans(n_clusters = 2).fit( X.drop(['ID_REGISTRO'], axis = 1) )

X['Labels'] = km2.labels_ 

plt.figure( figsize = (12, 8) )

sns.scatterplot( data = X, x = 'EDAD', y = 'Dias_Def', hue = 'Labels' )

plt.title('KMeans with 2 Clusters')

plt.show()

In [ ]:
# 3 cluster
km3 = KMeans(n_clusters = 3).fit( X.drop(['ID_REGISTRO'], axis = 1) )

X['Labels'] = km3.labels_ 

plt.figure( figsize = (12, 8) )

sns.scatterplot( data = X, x = 'EDAD', y = 'Dias_Def', hue = 'Labels' )

plt.title('KMeans with 3 Clusters')

plt.show()

In [ ]:
# 6 cluster
km6 = KMeans(n_clusters = 6).fit( X.drop(['ID_REGISTRO'], axis = 1) )

X['Labels'] = km6.labels_ 

plt.figure( figsize = (12, 8) )

sns.scatterplot( data = X, x = 'EDAD', y = 'Dias_Def', hue = 'Labels' )

plt.title('KMeans with 6 Clusters')

plt.show()

In [ ]:
#

X[ X['Labels'] == 0].describe()

In [ ]:
#

X[ X['Labels'] == 1].describe()

In [ ]:
#

X[ X['Labels'] == 2].describe()

In [ ]:
#

X[ X['Labels'] == 3].describe()

In [ ]:
#

X[ X['Labels'] == 4].describe()

In [ ]:
#

X[ X['Labels'] == 5].describe()